# Fairness  Behind  a  Veil  of  Ignorance adapted to group fairness

Import necessary dependencies.
Note that the modules "utils_modified" and "funcs_disp_mist_modified" containing some helper functions are adopted from "Fairness Beyond Disparate Treatment & Disparate Impact: Learning Classification without Disparate Mistreatment" by Zafar et. al. (https://github.com/mbilalzafar/fair-classification)

In [1]:
import sys
import numpy as np
import math
import scipy.io
import compas_preprocessing as preproc
import funcs_disp_mist_modified as fdm
from helpers import *
from cvxpy import *
from sklearn.model_selection import KFold
from sklearn.model_selection import ShuffleSplit

ModuleNotFoundError: No module named 'numpy'

## Importing the COMPAS dataset

A detailed description of the dataset can be found in the Appendix of the paper

Note that group 1 is white, group 0 is black

In [5]:
X, y, x_control = preproc.load_compas_data_modified()
sensitive_attrs = list(x_control.keys())

# since we want +1 to be the desired outcome we have to flip our training labels
# previously +1 indicated that a person recidivated within 2 years
y = -y

Looking for file 'compas-scores-two-years.csv' in the current directory...
File found in current directory..

Number of people recidivating within two years
-1    2795
 1    2483
dtype: int64


Features we will be using for classification are: ['intercept', 'age', 'juv_fel_count', 'juv_misd_count', 'juv_other_count', 'race', 'sex', 'priors_count', 'c_charge_degree'] 



Here we compute the maximum distance between any two data points in order to be able to compute the normalized distance

In [6]:
n = X.shape[0]
D = np.zeros((n,n))
for i in range(n):
    for j in range(n):
        D[i,j] = np.linalg.norm(X[i,:]- X[j,:])
d_max = D.max()

Setting some parameters for the solver

In [7]:
max_iters = 100 # for the convex program
max_iter_dccp = 50  # for the dccp algo
tau, mu = 0.005, 1.2 # default dccp parameters, need to be varied per dataset

The next two cells first split the data into train and validation set. Then it first trains an unconstrained classifier and a constrained classifier with specified parameters. This is useful for only running one particular test instead of iterating over alll tested configurations.

In [5]:
rs = ShuffleSplit(n_splits=1, test_size=.1, random_state=None)

for train_index, test_index in rs.split(X):
    x_train, x_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    x_control_train = {'race':x_control['race'][train_index]}
    x_control_test = {'race':x_control['race'][test_index]}

In [6]:
# set the alpha parameter for our utility function
alpha = -5
# threshold that average utility needs to be above, called tau in the plots  
threshold = -0.5

num_points, num_features = x_train.shape

# initialize weight vectors to a random value
w_uncons = Variable(num_features)
w_uncons.value = np.random.rand(x_train.shape[1])
w_cons = Variable(num_features)
w_cons.value = np.random.rand(x_train.shape[1])

print("Unconstrained:")

y_pred = (x_train*w_uncons)

# logistic loss
loss = sum_entries(logistic( mul_elemwise(-y_train, x_train*w_uncons) )  ) / num_points

constraints = [norm(w_uncons) == 1]

prob = Problem(Minimize(loss), constraints)

try:
    prob.solve(method='dccp', tau=tau, mu=mu, tau_max=1e10,
            solver=ECOS, verbose=False,
            max_iters=max_iters, max_iter=max_iter_dccp)
except Exception as e:
    print (e)
    print("check")


# convert solution vector to a numpy array
w_uncons = np.array(w_uncons.value).flatten()

train_score, test_score, cov_all_train, cov_all_test, s_attr_to_fp_fn_train, s_attr_to_fp_fn_test = fdm.get_clf_stats(w_uncons, x_train, y_train, x_control_train, x_test, y_test, x_control_test, sensitive_attrs)


print("Constrained:")    

y_pred = (x_train*w_cons)

# logistic loss
loss = sum_entries(logistic( mul_elemwise(-y_train, x_train*w_cons) )  ) / num_points


group_array_train = x_control_train['race']
# need to split predictions into groups to compute benefit separately
group0_filter = np.invert(group_array_train.astype(bool))
group1_filter = group_array_train.astype(bool)

n1 = sum(x_control_train['race'])
n0 = num_points - n1

y_pred1 = y_pred[group1_filter]
y_pred0 = y_pred[group0_filter]
y_train1 = y_train[group1_filter]
y_train0 = y_train[group0_filter]

bene1 = avg_benefit(y_pred1, y_train1)
bene0 = avg_benefit(y_pred0, y_train0)

if math.isinf(alpha):
    constraints = [threshold <= bene1,threshold <= bene0, norm(w_cons) == 1]
else:
    util = (n1*np.sign(alpha)*bene1**alpha + n0*np.sign(alpha)* bene0**alpha)/(2*num_points)

    # constraints for our optimization
    constraints = [threshold <= util, norm(w_cons) == 1]

prob = Problem(Minimize(loss), constraints)

solver_failed = False

try:
    prob.solve(method='dccp', tau=tau, mu=mu, tau_max=1e10,
            solver=ECOS, verbose=False,
            max_iters=max_iters, max_iter=max_iter_dccp)
except Exception as e:
    print (e)
    print("check")
    solver_failed = True

#convert solution vector to a numpy array
w_cons = np.array(w_cons.value).flatten()
print(prob.status)

if prob.status != "Converged" and prob.status != "OPTIMAL":
    solver_failed = True
    print("solver failed")
    
if not solver_failed:    
    y_test1 = (y_test[x_control_test['race'].astype(bool)])
    y_test0 = (y_test[np.invert(x_control_test['race'].astype(bool))])
    # achieved benefits and utilities of constrained classifier
    bene_cons1 = eval_benefit((np.dot(x_train,w_cons))[group1_filter], y_train1)
    bene_cons0 = eval_benefit((np.dot(x_train,w_cons))[group0_filter], y_train0)
    bene_cons_test1 = eval_benefit(((np.dot(x_test,w_cons)))[x_control_test['race'].astype(bool)], y_test1)
    bene_cons_test0 = eval_benefit(((np.dot(x_test,w_cons)))[np.invert(x_control_test['race'].astype(bool))], y_test0)
    util_cons = (n1*np.sign(alpha)*bene_cons1**alpha + n0*np.sign(alpha)* bene_cons0**alpha)/(2* num_points)

    y_pred_test = np.sign(np.dot(x_test, w_cons))
    y_proba_test = np.reciprocal(1+np.exp(np.dot(x_test, w_cons)))


    train_score, test_score, cov_all_train, cov_all_test, s_attr_to_fp_fn_train, s_attr_to_fp_fn_test = fdm.get_clf_stats(w_cons, x_train, y_train, x_control_train, x_test, y_test, x_control_test, sensitive_attrs)

    print("Benefit with constraints for group 0: " + str(bene_cons0) + "\n")
    print("Benefit with constraints for group 1: " + str(bene_cons1) + "\n")
    print("Benefit with constraints on test set for group 0: " + str(bene_cons_test0) + "\n")
    print("Benefit with constraints on test set for group 1: " + str(bene_cons_test1) + "\n")
    print("Utility with constraints: " + str(util_cons) + "\n")
    print("percentage of positively classified: " + str(np.sum(np.sign(np.dot(x_test,w_cons)))/y_test.shape[0]))

Unconstrained:
[ 1.  1.  1. ... -1.  1.  1.]


Accuracy: 0.642
||  s  || FPR. || FNR. ||
||  0  || 0.37 || 0.35 ||
||  1  || 0.62 || 0.19 ||


Constrained:
Converged
[ 1.  1.  1. ... -1.  1.  1.]


Accuracy: 0.640
||  s  || FPR. || FNR. ||
||  0  || 0.49 || 0.20 ||
||  1  || 0.73 || 0.16 ||


Benefit with constraints for group 0: 0.9623317063566134

Benefit with constraints for group 1: 1.081445692539817

Benefit with constraints on test set for group 0: 0.9490272026175999

Benefit with constraints on test set for group 1: 1.042696657360852

Utility with constraints: -0.5000000000552821

percentage of positively classified: 0.42045454545454547


### Generate cross validation splits for both training and evaulation

In [8]:
cross_val_iter = 10
kf = KFold(n_splits=cross_val_iter,shuffle=True)

## Here we train our classifier with different alpha and tau values and save the resulting weight vectors

Note that you need to run this part first before running the evaluation part. Also if the solver fails in one cross validation iteration the value of the weight vector is discarded. However, the computation for that only works after the first cross validation iteration. So if the solver does fail in the first iteration, you get a corresponding error message and need to rerun the training part.

In [9]:
# how many equispaced threshold values to try within the range specified later 
threshold_iter = 20
# from the feasible region we run it for some lower tau values to extend the plots
# of all alpha values to the left such that they all start at the same tau value
# those runs correspond to unconstrained classifiers, they just make the plots look nicer
left_extend_iter = 5

# number of features in the dataset
no_features = X.shape[1]


# this holds all the alpha values that we try
alpha_array = [-np.inf,-5,-1,0.1,0.2,0.4,0.8]

# how many different alpha parameter settings we try
no_params = len(alpha_array)

tau_inf = np.concatenate((np.linspace(-0.8,0.88,left_extend_iter),np.linspace(0.88,1.35,threshold_iter)))
tau_5 = np.concatenate((np.linspace(-0.8,-0.8,left_extend_iter),np.linspace(-0.8,-0.105,threshold_iter)))
tau_1 = np.concatenate((np.linspace(-0.8,-0.55,left_extend_iter),np.linspace(-0.55,-0.3655,threshold_iter)))
tau01 = np.concatenate((np.linspace(-0.8,0.49,left_extend_iter),np.linspace(0.49,0.5165,threshold_iter)))
tau02 = np.concatenate((np.linspace(-0.8,0.49,left_extend_iter),np.linspace(0.49,0.533,threshold_iter)))
tau04 = np.concatenate((np.linspace(-0.8,0.483,left_extend_iter),np.linspace(0.483,0.568,threshold_iter)))
tau08 = np.concatenate((np.linspace(-0.8,0.468,left_extend_iter),np.linspace(0.468,0.645,threshold_iter)))
# Holds all the tau values that we try depending on alpha
tau_matrix = np.column_stack((tau_inf,tau_5,tau_1,tau01,tau02,tau04,tau08))

# Here we will put the weights resulting from the training of the classifier
weight_matrix = np.zeros((left_extend_iter+threshold_iter,no_params,no_features))

i = 0
for train_index, test_index in kf.split(X):
    print("#############cross validation iteration " + str(i) + " #########################")
    x_train, x_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    x_control_train = {'race':x_control['race'][train_index]}
    x_control_test = {'race':x_control['race'][test_index]}
    num_points, num_features = x_train.shape

    alpha_counter = 0
    for alpha in alpha_array:
        print("#############Current alpha value: " + str(alpha) + " #########################")

        for j in range(0,left_extend_iter+threshold_iter):
            print("#############Threshold value iteration " + str(j) + " #########################")
            threshold = tau_matrix[j,alpha_counter]
            print("treshold: " + str(threshold))

            w_cons = Variable(num_features)
            w_cons.value = np.random.rand(x_train.shape[1])

            #logistic loss
            loss = sum_entries(logistic( mul_elemwise(-y_train, x_train*w_cons) )  ) / num_points # we are converting y to a diagonal matrix for consistent

            y_pred = (x_train*w_cons)

            group_array_train = x_control_train['race']
            # need to split predictions into groups to compute benefits separately
            group0_filter = np.invert(group_array_train.astype(bool))
            group1_filter = group_array_train.astype(bool)

            y_pred1 = y_pred[group1_filter]
            y_pred0 = y_pred[group0_filter]
            y_train1 = y_train[group1_filter]
            y_train0 = y_train[group0_filter]

            bene1 = avg_benefit(y_pred1, y_train1)
            bene0 = avg_benefit(y_pred0, y_train0)
            
            n1 = sum(x_control_train['race'])
            n0 = num_points - n1

            if math.isinf(alpha):
                constraints = [threshold <= bene1,threshold <= bene0, norm(w_cons) == 1]
            else:
                util = (n1*np.sign(alpha)*bene1**alpha + n0*np.sign(alpha)* bene0**alpha)/(2*num_points)

                # constraints for our optimization
                constraints = [threshold <= util, norm(w_cons) == 1]


            prob = Problem(Minimize(loss), constraints)

            solver_failed = False
            
            try:
                prob.solve(method='dccp', tau=tau, mu=mu, tau_max=1e10,
                        solver=ECOS, verbose=False,
                        max_iters=max_iters, max_iter=max_iter_dccp)
            except Exception as e:
                print (e)
                solver_failed = True


            if prob.status != "Converged" and prob.status != "OPTIMAL":
                solver_failed = True

            if not solver_failed: 
                w_cons = np.array(w_cons.value).flatten()

                weight_matrix[j,alpha_counter,:] += w_cons/cross_val_iter

                util_cons = eval_util((np.dot(x_train, w_cons)/5), y_train, alpha)

                train_score, test_score, cov_all_train, cov_all_test, s_attr_to_fp_fn_train, s_attr_to_fp_fn_test = fdm.get_clf_stats(w_cons, x_train, y_train, x_control_train, x_test, y_test, x_control_test, sensitive_attrs)
                print("Utility with constraints: " + str(util_cons) + "\n")
            else:
                # test if all values 0, which they are only in the first cross validation iteration
                if not np.any(weight_matrix[j,alpha_counter,:]):
                    sys.exit("Solver failed in first cross validation iteration")
                else:
                    weight_matrix[j,alpha_counter,:] = weight_matrix[j,alpha_counter,:]*(i + 1)/i
                
        alpha_counter += 1
    i += 1

#############cross validation iteration 0 #########################
#############Current alpha value: -inf #########################
#############Threshold value iteration 0 #########################
treshold: -0.8
[ 1. -1.  1. ... -1.  1.  1.]


Accuracy: 0.642
||  s  || FPR. || FNR. ||
||  0  || 0.37 || 0.35 ||
||  1  || 0.62 || 0.19 ||


Utility with constraints: -inf

#############Threshold value iteration 1 #########################
treshold: -0.38
[ 1. -1.  1. ... -1.  1.  1.]


Accuracy: 0.642
||  s  || FPR. || FNR. ||
||  0  || 0.37 || 0.35 ||
||  1  || 0.62 || 0.19 ||


Utility with constraints: -inf

#############Threshold value iteration 2 #########################
treshold: 0.040000000000000036
[ 1. -1.  1. ... -1.  1.  1.]


Accuracy: 0.642
||  s  || FPR. || FNR. ||
||  0  || 0.37 || 0.35 ||
||  1  || 0.62 || 0.19 ||


Utility with constraints: -inf

#############Threshold value iteration 3 #########################
treshold: 0.4600000000000002
[ 1. -1.  1. ... -1.  1.  1.

[ 1. -1.  1. ... -1.  1.  1.]


Accuracy: 0.638
||  s  || FPR. || FNR. ||
||  0  || 0.39 || 0.35 ||
||  1  || 0.62 || 0.19 ||


Utility with constraints: -21.07195135212687

#############Threshold value iteration 8 #########################
treshold: -0.6902631578947369
[ 1. -1.  1. ... -1.  1.  1.]


Accuracy: 0.646
||  s  || FPR. || FNR. ||
||  0  || 0.40 || 0.31 ||
||  1  || 0.63 || 0.18 ||


Utility with constraints: -19.962974930084815

#############Threshold value iteration 9 #########################
treshold: -0.6536842105263159
[ 1. -1.  1. ... -1.  1.  1.]


Accuracy: 0.646
||  s  || FPR. || FNR. ||
||  0  || 0.41 || 0.29 ||
||  1  || 0.64 || 0.18 ||


Utility with constraints: -18.902626148197445

#############Threshold value iteration 10 #########################
treshold: -0.6171052631578948
[ 1. -1.  1. ... -1.  1.  1.]


Accuracy: 0.644
||  s  || FPR. || FNR. ||
||  0  || 0.43 || 0.27 ||
||  1  || 0.66 || 0.17 ||


Utility with constraints: -17.888052009304683

#########

[ 1. -1.  1. ... -1.  1.  1.]


Accuracy: 0.640
||  s  || FPR. || FNR. ||
||  0  || 0.53 || 0.18 ||
||  1  || 0.80 || 0.09 ||


Utility with constraints: -1.3106005448165345

#############Threshold value iteration 14 #########################
treshold: -0.46260526315789474
[ 1. -1.  1. ... -1.  1.  1.]


Accuracy: 0.634
||  s  || FPR. || FNR. ||
||  0  || 0.59 || 0.14 ||
||  1  || 0.81 || 0.08 ||


Utility with constraints: -1.2999464674507706

#############Threshold value iteration 15 #########################
treshold: -0.4528947368421053
[ 1. -1.  1. ... -1.  1.  1.]


Accuracy: 0.631
||  s  || FPR. || FNR. ||
||  0  || 0.61 || 0.13 ||
||  1  || 0.85 || 0.06 ||


Utility with constraints: -1.2890768538273376

#############Threshold value iteration 16 #########################
treshold: -0.4431842105263158
[ 1.  1.  1. ... -1.  1.  1.]


Accuracy: 0.619
||  s  || FPR. || FNR. ||
||  0  || 0.65 || 0.11 ||
||  1  || 0.88 || 0.06 ||


Utility with constraints: -1.2779872829353738

#####

[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.604
||  s  || FPR. || FNR. ||
||  0  || 0.75 || 0.07 ||
||  1  || 0.91 || 0.04 ||


Utility with constraints: 0.9860362484270506

#############Threshold value iteration 20 #########################
treshold: 0.5109210526315789
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.602
||  s  || FPR. || FNR. ||
||  0  || 0.79 || 0.04 ||
||  1  || 0.93 || 0.01 ||


Utility with constraints: 0.9869088569475263

#############Threshold value iteration 21 #########################
treshold: 0.5123157894736842
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.589
||  s  || FPR. || FNR. ||
||  0  || 0.84 || 0.03 ||
||  1  || 0.94 || 0.01 ||


Utility with constraints: 0.9877848987419064

#############Threshold value iteration 22 #########################
treshold: 0.5137105263157894
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.574
||  s  || FPR. || FNR. ||
||  0  || 0.87 || 0.02 ||
||  1  || 1.00 || 0.01 ||


Utility with constraints: 0.988658337675798

#############Threshold value iteration

[ 1. -1.  1. ... -1.  1.  1.]


Accuracy: 0.642
||  s  || FPR. || FNR. ||
||  0  || 0.37 || 0.35 ||
||  1  || 0.62 || 0.19 ||


Utility with constraints: 0.926484604008534

#############Threshold value iteration 1 #########################
treshold: -0.47925000000000006
[ 1. -1.  1. ... -1.  1.  1.]


Accuracy: 0.642
||  s  || FPR. || FNR. ||
||  0  || 0.37 || 0.35 ||
||  1  || 0.62 || 0.19 ||


Utility with constraints: 0.9264846039559862

#############Threshold value iteration 2 #########################
treshold: -0.15850000000000009
[ 1. -1.  1. ... -1.  1.  1.]


Accuracy: 0.642
||  s  || FPR. || FNR. ||
||  0  || 0.37 || 0.35 ||
||  1  || 0.62 || 0.19 ||


Utility with constraints: 0.9264846039970541

#############Threshold value iteration 3 #########################
treshold: 0.1622499999999999
[ 1. -1.  1. ... -1.  1.  1.]


Accuracy: 0.642
||  s  || FPR. || FNR. ||
||  0  || 0.37 || 0.35 ||
||  1  || 0.62 || 0.19 ||


Utility with constraints: 0.9264846038697729

#############

...

[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.638
||  s  || FPR. || FNR. ||
||  0  || 0.62 || 0.08 ||
||  1  || 0.81 || 0.05 ||


Utility with constraints: -10.034464112293561

#############Threshold value iteration 20 #########################
treshold: -0.25131578947368416
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.622
||  s  || FPR. || FNR. ||
||  0  || 0.67 || 0.07 ||
||  1  || 0.82 || 0.04 ||


Utility with constraints: -9.169725424836102

#############Threshold value iteration 21 #########################
treshold: -0.21473684210526311
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.598
||  s  || FPR. || FNR. ||
||  0  || 0.73 || 0.07 ||
||  1  || 0.88 || 0.02 ||


Utility with constraints: -8.267530802969219

#############Threshold value iteration 22 #########################
treshold: -0.17815789473684207
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.583
||  s  || FPR. || FNR. ||
||  0  || 0.78 || 0.05 ||
||  1  || 0.93 || 0.01 ||


Utility with constraints: -7.313316831983465

#############Threshold value i

[ 1. -1.  1. ...  1.  1.  1.]


Accuracy: 0.706
||  s  || FPR. || FNR. ||
||  0  || 0.30 || 0.28 ||
||  1  || 0.59 || 0.12 ||


Utility with constraints: 0.9781952432415402

#############Threshold value iteration 1 #########################
treshold: -0.47750000000000004
[ 1. -1.  1. ...  1.  1.  1.]


Accuracy: 0.706
||  s  || FPR. || FNR. ||
||  0  || 0.30 || 0.28 ||
||  1  || 0.59 || 0.12 ||


Utility with constraints: 0.9781952433235338

#############Threshold value iteration 2 #########################
treshold: -0.15500000000000003
[ 1. -1.  1. ...  1.  1.  1.]


Accuracy: 0.706
||  s  || FPR. || FNR. ||
||  0  || 0.30 || 0.28 ||
||  1  || 0.59 || 0.12 ||


Utility with constraints: 0.9781952433060105

#############Threshold value iteration 3 #########################
treshold: 0.16749999999999998
[ 1. -1.  1. ...  1.  1.  1.]


Accuracy: 0.706
||  s  || FPR. || FNR. ||
||  0  || 0.30 || 0.28 ||
||  1  || 0.59 || 0.12 ||


Utility with constraints: 0.9781952433028404

###########

[ 1. -1.  1. ...  1.  1.  1.]


Accuracy: 0.706
||  s  || FPR. || FNR. ||
||  0  || 0.30 || 0.28 ||
||  1  || 0.59 || 0.12 ||


Utility with constraints: 0.9586304826531509

#############Threshold value iteration 7 #########################
treshold: 0.4945263157894737
[ 1. -1.  1. ...  1.  1.  1.]


Accuracy: 0.696
||  s  || FPR. || FNR. ||
||  0  || 0.31 || 0.28 ||
||  1  || 0.63 || 0.12 ||


Utility with constraints: 0.9592030635484721

#############Threshold value iteration 8 #########################
treshold: 0.4967894736842105
[ 1. -1.  1. ...  1.  1.  1.]


Accuracy: 0.694
||  s  || FPR. || FNR. ||
||  0  || 0.34 || 0.25 ||
||  1  || 0.64 || 0.11 ||


Utility with constraints: 0.9604236614668821

#############Threshold value iteration 9 #########################
treshold: 0.49905263157894736
[ 1. -1.  1. ...  1.  1.  1.]


Accuracy: 0.691
||  s  || FPR. || FNR. ||
||  0  || 0.37 || 0.21 ||
||  1  || 0.70 || 0.10 ||


Utility with constraints: 0.9616576690055231

#############Th

[ 1. -1.  1. ...  1.  1.  1.]


Accuracy: 0.658
||  s  || FPR. || FNR. ||
||  0  || 0.49 || 0.16 ||
||  1  || 0.81 || 0.05 ||


Utility with constraints: 0.9394454506438594

#############Threshold value iteration 13 #########################
treshold: 0.5187894736842105
[ 1. -1.  1. ...  1.  1.  1.]


Accuracy: 0.649
||  s  || FPR. || FNR. ||
||  0  || 0.53 || 0.14 ||
||  1  || 0.81 || 0.04 ||


Utility with constraints: 0.9417634256723363

#############Threshold value iteration 14 #########################
treshold: 0.5232631578947368
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.639
||  s  || FPR. || FNR. ||
||  0  || 0.60 || 0.09 ||
||  1  || 0.82 || 0.04 ||


Utility with constraints: 0.9440966331295247

#############Threshold value iteration 15 #########################
treshold: 0.5277368421052632
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.638
||  s  || FPR. || FNR. ||
||  0  || 0.62 || 0.09 ||
||  1  || 0.83 || 0.03 ||


Utility with constraints: 0.9464441520586391

#############Threshold va

[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.592
||  s  || FPR. || FNR. ||
||  0  || 0.75 || 0.06 ||
||  1  || 0.93 || 0.01 ||


Utility with constraints: 0.9310812697060931

#############Threshold value iteration 19 #########################
treshold: 0.598421052631579
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.584
||  s  || FPR. || FNR. ||
||  0  || 0.77 || 0.05 ||
||  1  || 0.94 || 0.01 ||


Utility with constraints: 0.9351624287239593

#############Threshold value iteration 20 #########################
treshold: 0.6077368421052631
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.571
||  s  || FPR. || FNR. ||
||  0  || 0.81 || 0.04 ||
||  1  || 0.96 || 0.01 ||


Utility with constraints: 0.9392443392274267

#############Threshold value iteration 21 #########################
treshold: 0.6170526315789474
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.567
||  s  || FPR. || FNR. ||
||  0  || 0.83 || 0.03 ||
||  1  || 0.98 || 0.00 ||


Utility with constraints: 0.9433234220412068

#############Threshold value iteration

### Saving the obtained weight vectors

In [10]:
scipy.io.savemat("../Result_Data/GroupFairness/W_matrix_group",{'dummykey':weight_matrix})

In [11]:
w04 = np.transpose(weight_matrix[left_extend_iter:,5,:])
scipy.io.savemat("../Result_Data/GroupFairness/weight_matrix0-4_group",{'dummykey':w04})

## Here we evaluate different metrics for all the weight vectors we got from the training process above

Note that we have to use the same validation splits to ensure testing is really done on validation data and not on data already seen during training.

In [12]:
# Here we will put in our results
# one column per parameter setting 

# holds the violations w.r.t. to the dwork constraints
dwork_matrix = np.zeros((left_extend_iter+threshold_iter,no_params))
# holds the fals positive rate
fpr_matrix = np.zeros((left_extend_iter+threshold_iter,no_params))
# false negative rate
fnr_matrix = np.zeros((left_extend_iter+threshold_iter,no_params))
# holds the absolute difference in percentage of positive outcomes
dp_matrix = np.zeros((left_extend_iter+threshold_iter,no_params))
# holds the atkinsons index
at_matrix = np.zeros((left_extend_iter+threshold_iter,no_params))
# holds ge2 unfairness
ge_matrix = np.zeros((left_extend_iter+threshold_iter,no_params))
# holds accuracy
accuracy_matrix = np.zeros((left_extend_iter+threshold_iter,no_params))

i = 0
for train_index, test_index in kf.split(X):
    print("#############cross validation iteration " + str(i) + " #########################")
    x_train, x_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    x_control_train = {'race':x_control['race'][train_index]}
    x_control_test = {'race':x_control['race'][test_index]}
    num_points, num_features = x_train.shape
    alpha_counter = 0
    for alpha in alpha_array:
        print("#############Current alpha value: " + str(alpha) + " #########################")

        for j in range(0,left_extend_iter+threshold_iter):
            print("#############Threshold value iteration " + str(j) + " #########################")
            threshold = tau_matrix[j,alpha_counter]
            print("treshold: " + str(threshold))

            w_cons = weight_matrix[j,alpha_counter,:]

            util_cons = eval_util((np.dot(x_train, w_cons)/5), y_train, alpha)

            train_score, test_score, cov_all_train, cov_all_test, s_attr_to_fp_fn_train, s_attr_to_fp_fn_test = fdm.get_clf_stats(w_cons, x_train, y_train, x_control_train, x_test, y_test, x_control_test, sensitive_attrs)

            y_pred_test = np.sign(np.dot(x_test, w_cons))

            group_array = x_control_test['race']
            
            # total number of positive and negative points for each group
            unique, counts = np.unique(y_test, return_counts=True)
            n_neg1 = 0
            n_pos1 = 0

            n_neg0 = 0
            n_pos0 = 0
            
            
            # total number of data points in test set
            n = y_test.shape[0]
            unique, counts = np.unique(group_array, return_counts=True)
            # number of people in group 0
            n0 = counts[0]
            # number of people in group 1
            n1 = counts[1]

            # positive prediction count for both classes and for predicted labels
            pos_count0_pred = 0
            pos_count1_pred = 0
            fp_count0 = 0
            fp_count1 = 0
            fn_count0 = 0
            fn_count1 = 0

            counter = 0
            for group_membership in group_array:
                if group_membership == 1:
                    if y_test[counter] == 1:
                        n_pos1 += 1
                    if y_test[counter] == -1:
                        n_neg1 += 1
                    # demographic parity check
                    if y_pred_test[counter] == 1:
                        pos_count1_pred += 1
                    #fpr fnr check
                    if y_test[counter] == 1 and y_pred_test[counter] == -1:
                        fn_count1 += 1
                    if y_test[counter] == -1 and y_pred_test[counter] == 1:
                        fp_count1 += 1
                elif group_membership == 0:
                    if y_test[counter] == 1:
                        n_pos0 += 1
                    if y_test[counter] == -1:
                        n_neg0 += 1
                    # demographic parity check
                    if y_pred_test[counter] == 1:
                        pos_count0_pred += 1
                    #fpr fnr check
                    if y_test[counter] == 1 and y_pred_test[counter] == -1:
                        fn_count0 += 1
                    if y_test[counter] == -1 and y_pred_test[counter] == 1:
                        fp_count0 += 1
                else:
                    print("ERROR")
                counter += 1


            print("Utility with constraints: " + str(util_cons) + "\n")
            dwork_matrix[j][alpha_counter] += avg_Dwork_violation(y_pred_test, x_test, 0, d_max) / cross_val_iter
            ge_matrix[j][alpha_counter] += ge2(y_pred_test, y_test) / cross_val_iter
            accuracy_matrix[j][alpha_counter] += test_score / cross_val_iter
            fpr_matrix[j][alpha_counter] += (fp_count0/n_neg0 - fp_count1/n_neg1) / cross_val_iter
            fnr_matrix[j][alpha_counter] += (fn_count0/n_pos0 - fn_count1/n_pos1) / cross_val_iter
            dp_matrix[j][alpha_counter] +=  (pos_count0_pred/n0 - pos_count1_pred/n1) / cross_val_iter
            if alpha < 0:
                at_matrix[j][alpha_counter] += 0
            else:
                at_matrix[j][alpha_counter] += atk(y_pred_test, y_test, alpha) / cross_val_iter
        alpha_counter += 1
    i += 1
dp_matrix = np.absolute(dp_matrix)
fpr_matrix = np.absolute(fpr_matrix)
fnr_matrix = np.absolute(fnr_matrix)

#############cross validation iteration 0 #########################
#############Current alpha value: -inf #########################
#############Threshold value iteration 0 #########################
treshold: -0.8
[ 1. -1.  1. ... -1.  1.  1.]


Accuracy: 0.699
||  s  || FPR. || FNR. ||
||  0  || 0.24 || 0.29 ||
||  1  || 0.60 || 0.20 ||


Utility with constraints: -inf

#############Threshold value iteration 1 #########################
treshold: -0.38
[ 1. -1.  1. ... -1.  1.  1.]


Accuracy: 0.699
||  s  || FPR. || FNR. ||
||  0  || 0.24 || 0.29 ||
||  1  || 0.60 || 0.20 ||


Utility with constraints: -inf

#############Threshold value iteration 2 #########################
treshold: 0.040000000000000036
[ 1. -1.  1. ... -1.  1.  1.]


Accuracy: 0.699
||  s  || FPR. || FNR. ||
||  0  || 0.24 || 0.29 ||
||  1  || 0.60 || 0.20 ||


Utility with constraints: -inf

#############Threshold value iteration 3 #########################
treshold: 0.4600000000000002
[ 1. -1.  1. ... -1.  1.  1.

#############Threshold value iteration 7 #########################
treshold: -0.726842105263158
[ 1. -1.  1. ... -1.  1.  1.]


Accuracy: 0.705
||  s  || FPR. || FNR. ||
||  0  || 0.25 || 0.26 ||
||  1  || 0.60 || 0.20 ||


Utility with constraints: -21.03228580518138

#############Threshold value iteration 8 #########################
treshold: -0.6902631578947369
[ 1. -1.  1. ... -1.  1.  1.]


Accuracy: 0.705
||  s  || FPR. || FNR. ||
||  0  || 0.27 || 0.23 ||
||  1  || 0.61 || 0.20 ||


Utility with constraints: -19.9818795392145

#############Threshold value iteration 9 #########################
treshold: -0.6536842105263159
[ 1. -1.  1. ... -1.  1.  1.]


Accuracy: 0.695
||  s  || FPR. || FNR. ||
||  0  || 0.31 || 0.21 ||
||  1  || 0.64 || 0.19 ||


Utility with constraints: -18.969957182282528

#############Threshold value iteration 10 #########################
treshold: -0.6171052631578948
[ 1. -1.  1. ... -1.  1.  1.]


Accuracy: 0.695
||  s  || FPR. || FNR. ||
||  0  || 0.34 |



Accuracy: 0.670
||  s  || FPR. || FNR. ||
||  0  || 0.47 || 0.13 ||
||  1  || 0.75 || 0.11 ||


Utility with constraints: -1.3241776715013271

#############Threshold value iteration 13 #########################
treshold: -0.47231578947368424
[ 1. -1.  1. ...  1.  1.  1.]


Accuracy: 0.665
||  s  || FPR. || FNR. ||
||  0  || 0.49 || 0.11 ||
||  1  || 0.76 || 0.10 ||


Utility with constraints: -1.313687896860344

#############Threshold value iteration 14 #########################
treshold: -0.46260526315789474
[ 1. -1.  1. ...  1.  1.  1.]


Accuracy: 0.657
||  s  || FPR. || FNR. ||
||  0  || 0.52 || 0.10 ||
||  1  || 0.80 || 0.09 ||


Utility with constraints: -1.302988107077731

#############Threshold value iteration 15 #########################
treshold: -0.4528947368421053
[ 1. -1.  1. ...  1.  1.  1.]


Accuracy: 0.642
||  s  || FPR. || FNR. ||
||  0  || 0.56 || 0.09 ||
||  1  || 0.82 || 0.09 ||


Utility with constraints: -1.2920722713196482

#############Threshold value iterati

#############Threshold value iteration 18 #########################
treshold: 0.5081315789473684
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.627
||  s  || FPR. || FNR. ||
||  0  || 0.63 || 0.05 ||
||  1  || 0.90 || 0.04 ||


Utility with constraints: 0.9849691962145124

#############Threshold value iteration 19 #########################
treshold: 0.5095263157894736
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.604
||  s  || FPR. || FNR. ||
||  0  || 0.70 || 0.05 ||
||  1  || 0.94 || 0.03 ||


Utility with constraints: 0.9858365788300657

#############Threshold value iteration 20 #########################
treshold: 0.5109210526315789
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.589
||  s  || FPR. || FNR. ||
||  0  || 0.74 || 0.04 ||
||  1  || 0.95 || 0.03 ||


Utility with constraints: 0.9867102434571436

#############Threshold value iteration 21 #########################
treshold: 0.5123157894736842
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.581
||  s  || FPR. || FNR. ||
||  0  || 0.78 || 0.02 ||
||  1  || 0.95

#############Threshold value iteration 24 #########################
treshold: 0.533
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.523
||  s  || FPR. || FNR. ||
||  0  || 0.98 || 0.00 ||
||  1  || 1.00 || 0.00 ||


Utility with constraints: 0.9812526530760921

#############Current alpha value: 0.4 #########################
#############Threshold value iteration 0 #########################
treshold: -0.8
[ 1. -1.  1. ... -1.  1.  1.]


Accuracy: 0.699
||  s  || FPR. || FNR. ||
||  0  || 0.24 || 0.29 ||
||  1  || 0.60 || 0.20 ||


Utility with constraints: 0.9257771697179338

#############Threshold value iteration 1 #########################
treshold: -0.47925000000000006
[ 1. -1.  1. ... -1.  1.  1.]


Accuracy: 0.699
||  s  || FPR. || FNR. ||
||  0  || 0.24 || 0.29 ||
||  1  || 0.60 || 0.20 ||


Utility with constraints: 0.9257771696893714

#############Threshold value iteration 2 #########################
treshold: -0.15850000000000009
[ 1. -1.  1. ... -1.  1.  1.]


Accuracy: 0.699
||  s  || 

...

#############Threshold value iteration 9 #########################
treshold: -0.6536842105263159
[ 1. -1.  1. ...  1.  1.  1.]


Accuracy: 0.647
||  s  || FPR. || FNR. ||
||  0  || 0.39 || 0.30 ||
||  1  || 0.65 || 0.19 ||


Utility with constraints: -18.859381161179982

#############Threshold value iteration 10 #########################
treshold: -0.6171052631578948
[ 1. -1.  1. ...  1.  1.  1.]


Accuracy: 0.645
||  s  || FPR. || FNR. ||
||  0  || 0.40 || 0.29 ||
||  1  || 0.65 || 0.19 ||


Utility with constraints: -17.885567562975684

#############Threshold value iteration 11 #########################
treshold: -0.5805263157894737
[ 1. -1.  1. ...  1.  1.  1.]


Accuracy: 0.641
||  s  || FPR. || FNR. ||
||  0  || 0.44 || 0.27 ||
||  1  || 0.65 || 0.19 ||


Utility with constraints: -16.94473556711681

#############Threshold value iteration 12 #########################
treshold: -0.5439473684210526
[ 1. -1.  1. ...  1.  1.  1.]


Accuracy: 0.647
||  s  || FPR. || FNR. ||
||  0  || 0



Accuracy: 0.653
||  s  || FPR. || FNR. ||
||  0  || 0.58 || 0.12 ||
||  1  || 0.78 || 0.05 ||


Utility with constraints: -1.3000439567590216

#############Threshold value iteration 15 #########################
treshold: -0.4528947368421053
[ 1. -1.  1. ...  1.  1.  1.]


Accuracy: 0.641
||  s  || FPR. || FNR. ||
||  0  || 0.62 || 0.12 ||
||  1  || 0.79 || 0.05 ||


Utility with constraints: -1.2891629448916306

#############Threshold value iteration 16 #########################
treshold: -0.4431842105263158
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.636
||  s  || FPR. || FNR. ||
||  0  || 0.65 || 0.10 ||
||  1  || 0.83 || 0.04 ||


Utility with constraints: -1.2780641782375313

#############Threshold value iteration 17 #########################
treshold: -0.43347368421052634
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.628
||  s  || FPR. || FNR. ||
||  0  || 0.68 || 0.08 ||
||  1  || 0.85 || 0.04 ||


Utility with constraints: -1.2667455472260964

#############Threshold value iteration 18 #####

#############Threshold value iteration 20 #########################
treshold: 0.5109210526315789
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.605
||  s  || FPR. || FNR. ||
||  0  || 0.78 || 0.03 ||
||  1  || 0.94 || 0.01 ||


Utility with constraints: 0.9869063412807365

#############Threshold value iteration 21 #########################
treshold: 0.5123157894736842
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.590
||  s  || FPR. || FNR. ||
||  0  || 0.83 || 0.02 ||
||  1  || 0.94 || 0.01 ||


Utility with constraints: 0.9877795268162401

#############Threshold value iteration 22 #########################
treshold: 0.5137105263157894
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.581
||  s  || FPR. || FNR. ||
||  0  || 0.88 || 0.01 ||
||  1  || 0.95 || 0.01 ||


Utility with constraints: 0.9886493311510502

#############Threshold value iteration 23 #########################
treshold: 0.5151052631578947
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.565
||  s  || FPR. || FNR. ||
||  0  || 0.93 || 0.00 ||
||  1  || 0.96



Accuracy: 0.651
||  s  || FPR. || FNR. ||
||  0  || 0.33 || 0.35 ||
||  1  || 0.61 || 0.21 ||


Utility with constraints: 0.9265788848442528

#############Threshold value iteration 1 #########################
treshold: -0.47925000000000006
[ 1. -1.  1. ...  1.  1.  1.]


Accuracy: 0.651
||  s  || FPR. || FNR. ||
||  0  || 0.33 || 0.35 ||
||  1  || 0.61 || 0.21 ||


Utility with constraints: 0.9265788848201421

#############Threshold value iteration 2 #########################
treshold: -0.15850000000000009
[ 1. -1.  1. ...  1.  1.  1.]


Accuracy: 0.651
||  s  || FPR. || FNR. ||
||  0  || 0.33 || 0.35 ||
||  1  || 0.61 || 0.21 ||


Utility with constraints: 0.9265788848119889

#############Threshold value iteration 3 #########################
treshold: 0.1622499999999999
[ 1. -1.  1. ...  1.  1.  1.]


Accuracy: 0.651
||  s  || FPR. || FNR. ||
||  0  || 0.33 || 0.35 ||
||  1  || 0.61 || 0.21 ||


Utility with constraints: 0.9265788848024811

#############Threshold value iteration 4 #



Accuracy: 0.645
||  s  || FPR. || FNR. ||
||  0  || 0.35 || 0.35 ||
||  1  || 0.62 || 0.21 ||


Utility with constraints: 0.8843304467143847

#############Threshold value iteration 7 #########################
treshold: 0.48663157894736847
[ 1. -1.  1. ...  1.  1.  1.]


Accuracy: 0.649
||  s  || FPR. || FNR. ||
||  0  || 0.39 || 0.30 ||
||  1  || 0.65 || 0.19 ||


Utility with constraints: 0.8883019209628759

#############Threshold value iteration 8 #########################
treshold: 0.49594736842105264
[ 1. -1.  1. ...  1.  1.  1.]


Accuracy: 0.641
||  s  || FPR. || FNR. ||
||  0  || 0.44 || 0.28 ||
||  1  || 0.72 || 0.14 ||


Utility with constraints: 0.89228551817968

#############Threshold value iteration 9 #########################
treshold: 0.5052631578947369
[ 1. -1.  1. ...  1.  1.  1.]


Accuracy: 0.653
||  s  || FPR. || FNR. ||
||  0  || 0.45 || 0.24 ||
||  1  || 0.73 || 0.11 ||


Utility with constraints: 0.8962805974305101

#############Threshold value iteration 10 ####

### Saving the obtained results

In [13]:
scipy.io.savemat("../Result_Data/GroupFairness/tau_matrix_group",{'dummykey':tau_matrix})
scipy.io.savemat("../Result_Data/GroupFairness/dwork_matrix_group",{'dummykey':dwork_matrix}) 
scipy.io.savemat("../Result_Data/GroupFairness/fpr_matrix_group",{'dummykey':fpr_matrix})
scipy.io.savemat("../Result_Data/GroupFairness/fnr_matrix_group",{'dummykey':fnr_matrix})
scipy.io.savemat("../Result_Data/GroupFairness/dp_matrix_group",{'dummykey':dp_matrix})
scipy.io.savemat("../Result_Data/GroupFairness/ge_matrix_group",{'dummykey':ge_matrix})
scipy.io.savemat("../Result_Data/GroupFairness/accuracy_matrix_group",{'dummykey':accuracy_matrix})
scipy.io.savemat("../Result_Data/GroupFairness/at_matrix_group",{'dummykey':at_matrix})